In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import requests
import random
import time
import argparse
from bs4 import BeautifulSoup

## Scraping the first page

In [21]:

r = requests.get('https://boardgamegeek.com/browse/boardgame/page/1')

html = r.text
soup = BeautifulSoup(html, 'html.parser')

In [27]:
#print(soup.body.prettify())

In [29]:
soup.find_all('Robot.txt')

[]

In [22]:
rows = soup.body.find_all('tr', attrs={'id':'row_'})


In [101]:
ranks = [row.find('td', attrs={'class':'collection_rank'}).get_text().strip() for row in rows]
no_ratings = [row.find_all('td', attrs={'class':'collection_bggrating'})[2].get_text().strip() for row in rows]
names = [row.find('a', attrs={'class':'primary'}).get_text() for row in rows]
links = [row.find('a', attrs={'class':'primary'}).get('href') for row in rows]
ids = [int(row.find('a', attrs={'class':'primary'}).get('href').split('/')[2]) for row in rows]

In [104]:
df = pd.DataFrame({
    'rank':ranks,
    'name':names,
    'id':ids,
    'links':links,
    'number_of_ratings':no_ratings
})

In [105]:
df

,rank,name,id,links,number_of_ratings
0,1,Gloomhaven,174430,/boardgame/174430/gloomhaven,44335
1,2,Pandemic Legacy: Season 1,161936,/boardgame/161936/pandemic-legacy-season-1,42998
2,3,Brass: Birmingham,224517,/boardgame/224517/brass-birmingham,21602
3,4,Terraforming Mars,167791,/boardgame/167791/terraforming-mars,68378
4,5,Twilight Imperium: Fourth Edition,233078,/boardgame/233078/twilight-imperium-fourth-edi...,14376
...,...,...,...,...,...
95,96,Lorenzo il Magnifico,203993,/boardgame/203993/lorenzo-il-magnifico,9762
96,97,Codenames,178900,/boardgame/178900/codenames,70110
97,98,The Isle of Cats,281259,/boardgame/281259/isle-cats,8320
98,99,Roll for the Galaxy,132531,/boardgame/132531/roll-galaxy,27126


## automated scraping

In [34]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

boardgames = pd.DataFrame()
page = 1
num_voters = 100

while num_voters > 9:
    print(page)
    url = f'https://boardgamegeek.com/browse/boardgame/page/{page}?sort=numvoters&sortdir=desc'
    #url = f'https://boardgamegeek.com/browse/boardgame/page/{page}'
    r = requests.get(url, headers=headers)
    html = r.text
    soup = BeautifulSoup(html, 'html.parser')
    
    rows = soup.body.find_all('tr', attrs={'id':'row_'})
    ranks = [row.find('td', attrs={'class':'collection_rank'}).get_text().strip() for row in rows]
    no_ratings = [int(row.find_all('td', attrs={'class':'collection_bggrating'})[2].get_text().strip()) for row in rows]
    names = [row.find('a', attrs={'class':'primary'}).get_text() for row in rows]
    links = [row.find('a', attrs={'class':'primary'}).get('href') for row in rows]
    ids = [int(row.find('a', attrs={'class':'primary'}).get('href').split('/')[2]) for row in rows]
    
    df = pd.DataFrame({
        'rank':ranks,
        'name':names,
        'id':ids,
        'links':links,
        'num_voters':no_ratings
    })
    
    boardgames = boardgames.append(df, ignore_index=True)
    num_voters = boardgames['num_voters'].min()
    
    if page%10==0:
        boardgames.to_csv('../data/boardgames.csv', index=False)
    
    page = page + 1
    time.sleep(3)
    
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [20]:
random.random()/10

0.03057819781329204

In [18]:
page = 350

In [19]:
page%50

0

In [41]:
boardgames.sort_values('id', ascending=True).head(20)

,rank,name,id,links,num_voters
696,297,Die Macher,1,/boardgame/1/die-macher,5271
5041,3841,Dragonmaster,2,/boardgame/2/dragonmaster,558
209,217,Samurai,3,/boardgame/3/samurai,14925
7099,5109,Tal der Könige,4,/boardgame/4/tal-der-konige,338
150,280,Acquire,5,/boardgame/5/acquire,18368
16955,10140,Mare Mediterraneum,6,/boardgame/6/mare-mediterraneum,82
1117,1916,Cathedral,7,/boardgame/7/cathedral,3269
10027,8706,Lords of Creation,8,/boardgame/8/lords-creation,200
2488,2802,El Caballero,9,/boardgame/9/el-caballero,1382
445,1078,Elfenland,10,/boardgame/10/elfenland,8178
